In [8]:
%useLatestDescriptors
%use lets-plot
%use dataframe
%use kandy

In [2]:
import org.litote.kmongo.serialization.SerializationClassMappingTypeService
import com.mongodb.client.MongoClients
import org.litote.kmongo.withKMongo
import java.io.File
import java.util.*

System.setProperty(
    "org.litote.mongo.mapping.service",
    SerializationClassMappingTypeService::class.qualifiedName!!
)

val properties = Properties()
properties.load(
    File("G:\\Organizations\\Duels\\duels-datascience\\dfl.properties").reader()
)

val mongoConnectionUri = properties.getProperty("mongo.clients.uri")
val mongo = MongoClients.create(mongoConnectionUri)

val database = mongo.getDatabase("duels_anticheat").withKMongo()


In [18]:
import kotlinx.serialization.Contextual
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import org.bson.types.ObjectId
import org.litote.kmongo.*
import java.text.SimpleDateFormat
import java.time.*
import kotlin.collections.count

@Serializable
data class AnticheatLog(
    @SerialName("_id")
    val _id: @Contextual ObjectId,
    val timestamp: Long,
    val ping: Long,
    val vl: Int,
    val check: String
)

val startOfToday = LocalDate
    .now(ZoneId.systemDefault())
    .atTime(LocalTime.MIDNIGHT)
    .atZone(ZoneId.systemDefault())
    
val formatter = SimpleDateFormat("hh.mm a")
    
val logs = database.getCollection("logs").withKMongo()
val result = logs.aggregate<AnticheatLog>(
    match(
        AnticheatLog::check eq "Double Click A",
        AnticheatLog::timestamp gte (startOfToday.toInstant().toEpochMilli() - Duration.ofHours(24).toMillis()),
        AnticheatLog::timestamp lte (startOfToday.toInstant().toEpochMilli()),
    )
).toList()

val earliestTimestamp = result.minOfOrNull { it.timestamp } ?: Instant.now().toEpochMilli()
val latestTimestamp = result.maxOfOrNull { it.timestamp } ?: Instant.now().toEpochMilli()

val intervalDuration = Duration.ofMinutes(5).toMillis()
val bins = (earliestTimestamp..latestTimestamp) step intervalDuration

val violationsCountInInterval = bins.associateWith { binStart ->
    val binEnd = binStart + intervalDuration
    result.count { log ->
        log.timestamp in binStart until binEnd
    }
}

val intervals by columnOf(
    *violationsCountInInterval.keys
        .map { formatter.format(Date(it)) }
        .toTypedArray()
)

val violations by columnOf(
    *violationsCountInInterval.values.toTypedArray()
)

println(intervals.countDistinct())
println(violations.countDistinct())

dataFrameOf(intervals, violations)
    .plot {
        bars {
            x(intervals)
            y(violations)
        }
        
        layout {
            title = "Double Clicking Violations in the last 24 Hours"
            size = 900 to 550
            
            xAxisLabel = "Intervals"
            yAxisLabel = "Violations"
        }
    }

90
34


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iAVpAz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Double Clicking Violations in the last 24 Hours"
},
"mapping":{
},
"data":{
"intervals":["03.32 PM","03.37 PM","03.42 PM","03.47 PM","03.52 PM","03.57 PM","04.02 PM","04.07 PM","04.12 PM","04.17 PM","04.22 PM","04.27 PM","04.32 PM","04.37 PM","04.42 PM","04.47 PM","04.52 PM","04.57 PM","05.02 PM","05.07 PM","05.12 PM","05.17 PM","05.22 PM","05.27 PM","05.32 PM","05.37 PM","05.42 PM","05.47 PM","05.52 PM","05.57 PM","06.02 PM","06.07 PM","06.12 PM","06.17 PM","06.22 PM","06.27 PM","06.32 PM","06.37 PM","06.42 PM","06.47 PM","06.52 PM","06.57 PM","07.02 PM","07.07 PM","07.12 PM","07.17 PM","07.22 PM","07.27 PM","07.32 PM","07.37 PM","07.42 PM","07.47 PM","07.52 PM","07.57 PM","08.02 PM","08.07 PM","08.12 PM","08.17 PM","08.22 PM","08.27 PM","08.32 PM","08.37 PM","08.42 PM","08.47 PM","08.52 PM","08.57 PM","09.02 PM","09.07 PM","09.12 PM","09.17 PM","09.22 PM","09.27 PM","09.32 PM","09.37 PM","09.42 PM","09.47 PM","09.52 PM","09.57 PM","10.02 PM","10.07 PM","10.12 PM","10.17 PM","10.22 PM","10.27 PM","10.32 PM","10.37 PM","10.42 PM","10.47 PM","10.52 PM","10.57 PM"],
"violations":[10.0,38.0,25.0,103.0,61.0,136.0,171.0,178.0,35.0,29.0,37.0,28.0,29.0,30.0,33.0,34.0,35.0,35.0,120.0,109.0,27.0,38.0,128.0,63.0,49.0,86.0,72.0,69.0,52.0,50.0,76.0,24.0,44.0,21.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,35.0,26.0,3.0]
},
"ggsize":{
"width":900.0,
"height":550.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Intervals"
},{
"aesthetic":"y",
"name":"Violations"
}],
"layers":[{
"mapping":{
"x":"intervals",
"y":"violations"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("iAVpAz");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 03.32 PM
 
 
 
 
 
 
 
 
 
 
 
 
 03.42 PM
 
 
 
 
 
 
 
 
 
 
 
 
 03.52 PM
 
 
 
 
 
 
 
 
 
 
 
 
 04.02 PM
 
 
 
 
 
 
 
 
 
 
 
 
 04.12 PM
 
 
 
 
 
 
 
 
 
 
 
 
 04.22 PM
 
 
 
 
 
 
 
 
 
 
 
 
 04.32 PM
 
 
 
 
 
 
 
 
 
 
 
 
 04.42 PM
 
 
 
 
 
 
 
 
 
 
 
 
 04.52 PM
 
 
 
 
 
 
 
 
 
 
 
 
 05.02 PM
 
 
 
 
 
 
 
 
 
 
 
 
 05.12 PM
 
 
 
 
 
 
 
 
 
 
 
 
 05.22 PM
 
 
 
 
 
 
 
 
 
 
 
 
 05.32 PM
 
 
 
 
 
 
 
 
 
 
 
 
 05.42 PM
 
 
 
 
 
 
 
 
 
 
 
 
 05.52 PM
 
 
 
 
 
 
 
 
 
 
 
 
 06.02 PM
 
 
 
 
 
 
 
 
 
 
 
 
 06.12 PM
 
 
 
 
 
 
 
 
 
 
 
 
 06.22 PM
 
 
 
 
 
 
 
 
 
 
 
 
 06.32 PM
 
 
 
 
 
 
 
 
 
 
 
 
 06.42 PM
 
 
 
 
 
 
 
 
 
 
 
 
 06.52 PM
 
 
 
 
 
 
 
 
 
 
 
 
 07.02 PM
 
 
 
 
 
 
 
 
 
 
 
 
 07.12 PM
 
 
 
 
 
 
 
 
 
 
 
 
 07.22 PM
 
 
 
 
 
 
 
 
 
 
 
 
 07.32 PM
 
 
 
 
 
 
 
 
 
 
 
 
 07.42 PM
 
 
 
 
 
 
 
 
 
 
 
 
 07.52 PM
 
 
 
 
 
 
 
 
 
 
 
 
 08.02 PM
 
 
 
 
 
 
 
 
 
 
 
 
 08.12 PM
 
 
 
 
 
 
 
 
 
 
 
 
 08.22 PM
 
 
 
 
 
 
 
 
 
 
 
 
 08.32 PM
 
 
 
 
 
 
 
 
 
 
 
 
 08.42 PM
 
 
 
 
 
 
 
 
 
 
 
 
 08.52 PM
 
 
 
 
 
 
 
 
 
 
 
 
 09.02 PM
 
 
 
 
 
 
 
 
 
 
 
 
 09.12 PM
 